In [1]:
%ls

 LICENSE                             package-lock.json
'Notebook for country club .ipynb'   script.py
 README.md                           sqlite_db_pythonsqlite.db


In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [4]:
import pandas as pd

In [5]:
# Import tables as DataFrames
members = pd.read_sql_query('SELECT * FROM Members', engine)
bookings = pd.read_sql_query('SELECT * FROM Bookings', engine)
facilities = pd.read_sql_query('SELECT * FROM Facilities', engine)


In [6]:
# /* Q10: Produce a list of facilities with a total revenue less than 1000.
# The output of facility name and total revenue, sorted by revenue. Remember
# that there's a different cost for guests and members! */

query_string = """
WITH 
    facilities_bookings AS
        (
        SELECT 
            name AS facility_name,
            membercost,
            guestcost,
            memid
        FROM Bookings
            INNER JOIN
            Facilities USING (facid)
        ),
    guest_facilities_bookings AS
        (
        SELECT 
            facility_name,
            guestcost AS cost,
            memid
        FROM facilities_bookings
        WHERE memid = 0
        ),
    member_facilities_bookings AS
        (
        SELECT 
            facility_name,
            membercost AS cost,
            memid
        FROM facilities_bookings
        WHERE memid != 0
        ),
    guest_facility_revenue AS 
        (
        SELECT 
            facility_name,
            SUM(cost) AS revenue_from_guests
        FROM guest_facilities_bookings
        GROUP BY facility_name
        ),
    member_facility_revenue AS 
        (
        SELECT 
            facility_name,
            SUM(cost) AS revenue_from_members
        FROM member_facilities_bookings
        GROUP BY facility_name
        ),
    facilities_revenue AS
        (
        SELECT *
        FROM guest_facility_revenue 
            JOIN 
            member_facility_revenue USING (facility_name)
        )
SELECT 
    facility_name,
    (revenue_from_guests + revenue_from_members) AS total_revenue
FROM facilities_revenue
WHERE total_revenue < 1000;
"""

facs_with_total_rev_under_1000 = pd.read_sql_query(query_string, engine)

In [7]:
facs_with_total_rev_under_1000

,facility_name,total_revenue
0,Badminton Court,604.5
1,Pool Table,265.0
2,Snooker Table,115.0
3,Table Tennis,90.0


In [8]:

# /* Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */

sqlite_query_string = """
SELECT 
    m1.surname || ', ' || m1.firstname AS member,
    m2.surname || ', ' || m2.firstname AS referrer
FROM Members AS m1
    JOIN
    Members AS m2 
        ON m1.recommendedby = m2.memid
ORDER BY member;
"""

mysql_query_string = """
SELECT 
    CONCAT(m1.surname, ', ', m1.firstname) AS member,
    concat(m2.surname, ', ', m2.firstname) AS referrer
FROM Members AS m1
    JOIN
    Members AS m2 
        ON m1.recommendedby = m2.memid
ORDER BY member;
"""

member_referrers = pd.read_sql_query(sqlite_query_string, engine)

In [9]:
member_referrers

,member,referrer
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


In [19]:
# /* Q12: Find the facilities with their usage by member, but not guests */

sqlite_query_string = """
WITH 
    mem_fac_book AS
        (
        SELECT *
        FROM Bookings
            INNER JOIN
            Facilities USING (facid)
            INNER JOIN
            Members USING (memid)
        ),
    mem_fac_book_ex_guests AS
        (
        SELECT * 
        FROM mem_fac_book
        WHERE memid != 0
        ),
    facilities_usage_ex_guests_by_member_and_facility AS
        (
        SELECT 
            name AS facility,
            surname || ', ' || firstname AS member,
            COUNT(*) AS "usage (by number of bookings)",
            COUNT(*)*membercost AS "usage (by costs paid)"
        FROM mem_fac_book_ex_guests
        GROUP BY facility, member
        )
SELECT *
FROM facilities_usage_ex_guests_by_member_and_facility;
"""

facilities_with_usage_ex_guests_by_member = pd.read_sql_query(sqlite_query_string, engine)
facilities_with_usage_ex_guests_by_member

,facility,member,usage (by number of bookings),usage (by costs paid)
0,Badminton Court,"Bader, Florence",9,0.0
1,Badminton Court,"Baker, Anne",10,0.0
2,Badminton Court,"Baker, Timothy",7,0.0
3,Badminton Court,"Boothe, Tim",12,0.0
4,Badminton Court,"Butters, Gerald",20,0.0
...,...,...,...,...
197,Tennis Court 2,"Smith, Darren",19,95.0
198,Tennis Court 2,"Smith, Jack",1,5.0
199,Tennis Court 2,"Smith, Tracy",2,10.0
200,Tennis Court 2,"Stibbons, Ponder",31,155.0


In [21]:
# /* Q13: Find the facilities usage by month, but not guests */

sqlite_query_string = """
WITH 
    mem_fac_book AS
        (
        SELECT *
        FROM Bookings
            INNER JOIN
            Facilities USING (facid)
            INNER JOIN
            Members USING (memid)
        ),
    mem_fac_book_ex_guests AS
        (
        SELECT * 
        FROM mem_fac_book
        WHERE memid != 0
        ),
    facilities_usage_ex_guests_by_month AS
        (
        SELECT 
            STRFTIME('%m', starttime) AS month,
            COUNT(*) AS "usage (by number of bookings)",
            COUNT(*)*membercost AS "usage (by costs paid)"
        FROM mem_fac_book_ex_guests
        GROUP BY month
        )
SELECT *
FROM facilities_usage_ex_guests_by_month;
"""

facilities_with_usage_ex_guests_by_month = pd.read_sql_query(sqlite_query_string, engine)
facilities_with_usage_ex_guests_by_month

,month,usage (by number of bookings),usage (by costs paid)
0,07,480,0
1,08,1168,5840
2,09,1512,7560
